In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.dummy import DummyRegressor

import eli5
import catboost

import mlflow
import missingno as msno
import gc
from os.path import exists
from tqdm import tqdm

pd.set_option('display.max_columns', None)

### Defining function used for experiments

In [53]:
def get_or_create_experiment(experiment_name):
    """Checks if mlflow experiment already exist (if not creates it) and return it."""
    experiment = mlflow.get_experiment_by_name(experiment_name)
    if not experiment:
        mlflow.create_experiment(experiment_name)
        return mlflow.get_experiment_by_name(experiment_name)        
    return experiment


def _eid(experiment_name):
    return get_or_create_experiment(experiment_name).experiment_id


def get_filename(basename):
    """Creates unique filename to prevent overwriting files in target directory when exporting the forecast."""
    counter=0
    filename = '{}_{}'.format(basename, counter)  # filename in format <basename>_<number>.csv
    # Adds +1 to <number> until it gets unique filename
    while exists('output/{}.csv'.format(filename)):
        counter +=1
        filename = '{}_{}'.format(basename, counter)        
    return filename


def get_X_y_log(df, feats):
    """Creates feature matrix, target and logarihtmic vector for model training."""
    X = df[feats]
    y = df['price']
    y_log = np.log(y)    
    return X, y, y_log
    

def get_feats(df, blacklist_extended):
    """Creates training features by taking all number and boolean columns 
    from provided dataframe, reduced by those in blacklist.
    """    
    blacklist = [
        'price',
        'id', 
        'price_m2',
    ]
    # Extra blacklist feature can be added as list in function argument. 
    if blacklist_extended:
        blacklist = blacklist + blacklist_extended       
                         
    num_bool_feats = df.select_dtypes(['number', 'bool'])
    return [x for x in num_bool_feats if x not in blacklist]


def get_model(model_or_id):
    """Returns one of two default models, or alternative one, provided as function argument."""
    if model_or_id == 'catboost_hyper':  # hyper parameters tuned for better forecast, but longer model training.
        model_params = dict(             
            max_depth=8,
            n_estimators=1000,
            learning_rate=0.3,
            random_state=0,
            silent=True,
        )
        return catboost.CatBoostRegressor(**model_params)
    
    elif model_or_id == 'catboost_light':  # hyper parameters tuned for quick results.
        model_params = dict(            
            max_depth=5,
            n_estimators=100,
            random_state=0,
            silent=True,
        )
        return catboost.CatBoostRegressor(**model_params)
    else:
        return model_or_id


def mlflow_experiment(run_name, model, X, feats, result, eli5_result):
    """Uses MLflow tool to record and track important information about experiments.
    Such as model parameters, shape of feature and targe matrix, feats used for model training or eli5 importance of the features
    """
    with mlflow.start_run(experiment_id=_eid('dw_solution_property'), run_name=run_name) as run:
    
        mlflow.log_params(model.get_params())
        mlflow.log_param("model", str(model).split("(")[0])
        mlflow.log_param("feats", feats)
        mlflow.log_param('X.shape', X.shape)
        
        # artifacts
        with open('outputs/eli5.html', 'w') as f:
            f.write('<html>{}</html>'.format(eli5_result.data))
        mlflow.log_artifact('outputs/eli5.html', 'plot')
                
        # metrics
        mlflow.log_metric('mae_mean', result[0])
        mlflow.log_metric('mae_std', result[1])
              
    print(f'Experiment {run_name} recorded')

    
def check_model(X, y, model, scoring):
    """Evaluate model based on selected metric."""
    cv = KFold(n_splits=5, shuffle=True, random_state=0)
    scores = []    
    for train_idx, test_idx in tqdm(cv.split(X)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_log_train, y_test = np.log(y).iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_log_train)
        y_log_pred = model.predict(X_test)
        y_pred = np.exp(y_log_pred)

        score = scoring(y_test, y_pred)
        scores.append(score)
        
    return (np.mean(scores), np.std(scores))
    
    
def save_forecast_to_csv(model, X_test, df_test, filename):
    """Exports forecast to csv file"""
    y_pred = model.predict(X_test)
    df_test['price'] = np.exp(y_pred)
    
    try:
        df_test[ ['id', 'price'] ].to_csv('output/{}.csv'.format(filename), index=False)
        print('{}.csv save successfully in "output" folder'.format(filename))
    except:
        print('{}.csv saving ERROR in "output" folder'.format(filename))
    
    
def start_experiment(df, model_or_id, scoring=mean_absolute_error,
                     filename='catboost',
                     results=True, export=False, mlflow_save=True, blacklist_extended=None):
    """Main function to conducts experiment.
    
    Main feature are:
    record experiment data using MLflow - "mlflow_save" argument,
    showing results based on selected metric - "results" argument,
    exporting forecast to csv file - "export" argument.
    All above argument are boolean and can be set independently according to expected output.
    Mlflow_save argument can be set to True only if results argument is also set to True.
    """
    if mlflow_save:
        assert results, '"mlflow_save" argument can be set to True only if "results" argument is also set to True.'
        
    filename = get_filename(filename)    
    print(f'Working on "{filename}" experiment')
    
    # Splits dataset into train and test dataframes.     
    df_train = df[ df['price'].notnull() ].fillna(-1)  # takes all rows with "price" column NOT empty
    df_test = df[ df['price'].isnull() ].fillna(-1)  # # takes all rows with "price" column empty
    
    # Gets X and y (feature matrix and target vector). 
    feats = get_feats(df, blacklist_extended )
    X, y, y_log = get_X_y_log(df_train, feats)

    # Gets model
    model = get_model(model_or_id)
    
    # Returning results
    if results:
        result = check_model(X, y, model, scoring)
        
    model.fit(X.values, y_log.values)
    
    # Export forecast to csv.
    if export:
        X_test = df_test[feats].values
        save_forecast_to_csv(model, X_test, df_test, filename)    
  
    eli5_result = eli5.show_weights(model, feature_names=feats)
    
    #MLflow tracking
    if mlflow_save and results:
        mlflow_experiment(filename, model, X, feats, result, eli5_result)

    if results:
        print(result)
    return eli5_result  

## Load data

In [ ]:
df_train = pd.read_hdf('data/train_property.h5')
df_test = pd.read_hdf('data/test_property.h5')
df_org = pd.concat([df_train, df_test])

# External city and province data (Wikipedia)
city_stats = pd.read_csv('external_data/city_stats_wiki.csv')
city_stats.drop('Województwo', axis=1, inplace=True)
city_stats.columns = ['city', 'county', 'city_area', 'city_population', 'city_density']

province_stats = pd.read_csv('external_data/province_stats_wiki.csv')
province_stats.drop('Lp.', axis=1, inplace=True)
province_stats.columns = ['province', 'province_population', 'province_men_population', 'province_women_population']

print(df_train.shape, df_test.shape, df_org.shape)

del df_train, df_test
gc.collect

## Feature engineering

In [16]:
def feature_engineering(df):
    
    def parse_czynsz(val):
        if isinstance(val, int): return val
        if isinstance(val, float): return val

        if val[-1] == 'ł':
            return float(val.split('zł')[0].replace(' ', '').replace(',','.'))
        if val[-1] == 'r':
            return float(val.split('eur')[0].replace(' ', '').replace(',','.'))*4.5
    
    
    def parse_area(val):
        if isinstance(val, int): return val
        if isinstance(val, float): return val

        return float(val.split('m')[0].replace(',','.').replace(' ',''))
    

    def parse_location_city(val):
        all_city = city_stats['city'].to_list()
        for city_ in reversed(val):
            if city_ in ['Dobra', 'Józefów']:
                continue
            if city_ in all_city:
                return city_
        return 'other'    
    

    def parse_floors_in_building(val):
        if isinstance(val, int): return -1
        if isinstance(val, float): return -1    
        floor = float(val.replace(')','').split()[1])
        return floor if floor < 20 else 25
    
    def df_groupby_feat(df, groupby_feats, feat):
        agg_params={
            'mean_{}_{}'.format('_'.join(groupby_feats), feat): (feat, 'mean'),
            'median_{}_{}'.format('_'.join(groupby_feats), feat): (feat, 'median'),
            #'sum_{}_price'.format('_'.join(groupby_feats)): ('price', 'sum')
        }
        return df[groupby_feats + [feat]].groupby(groupby_feats).agg(
            **agg_params
        ).reset_index()

    df['czynsz_num'] = df.czynsz.map(parse_czynsz)
    
    df['area_num'] = df.area.map(parse_area)
    area_num_99 = np.percentile(df['area_num'], 99)
    df['area_norm'] = df['area_num'].map(lambda x: x if x <= area_num_99 else area_num_99)
    df['area_num_log'] = np.log(df['area_num'])
    
    df['price_m2'] = df['price'] / df['area_num']
    
    df['province'] = df['location'].map(lambda x: x[0])
    df['city'] = df['location'].map(parse_location_city)
    df['floors_in_building_num'] = df['floors_in_building'].map(parse_floors_in_building)
    df['build_year'] = df['rok budowy'].fillna(-1).astype('int')
    
    floors_dict = {'parter': 0, '> 10': 11, 'poddasze': -2, 'suterena': -1}
    df['floor_num'] = df['floor'].map(lambda x: floors_dict.get(x, x)).fillna(-10).astype('int')
    
    
    if 'city_area' not in df.columns:
        df = pd.merge(df, city_stats, on='city', how='left')
    if 'province_population' not in df.columns:
        df = pd.merge(df, province_stats, on='province', how='left')
    
    miasta_woj = ['Białystok', 'Bydgoszcz', 'Gdańsk', 'Gorzów Wielkopolski', 'Katowice', 'Kielce', 'Kraków', 'Lublin',
    'Łódź', 'Olsztyn', 'Opole', 'Poznań', 'Rzeszów', 'Szczecin', 'Toruń', 'Warszawa', 'Wrocław', 'Zielona Góra']
    
    df['miasta_woj'] = df['city'].isin(miasta_woj)
    
    #Agregacje
    groupby_city_price = df_groupby_feat(df, ['city'], 'price')        
    if 'median_city_price' not in df:
        df = pd.merge(df, groupby_city_price, on='city', how='left')
        
    groupby_county_price = df_groupby_feat(df, ['county'], 'price')        
    if 'median_county_price' not in df:
        df = pd.merge(df, groupby_county_price, on='county', how='left')
    
    #groupby_city_pricem2 = df_groupby_feat(df, ['city'], 'price_m2')        
    #if 'median_county_price_m2' not in df:
    #    df = pd.merge(df, groupby_city_pricem2, on='city', how='left')
        
    #groupby_county_pricem2 = df_groupby_feat(df, ['county'], 'price_m2')        
    #if 'median_county_price_m2' not in df:
    #    df = pd.merge(df, groupby_county_pricem2, on='county', how='left')
        
    #groupby_province_price = df_group_price(df, ['province'])        
    #if 'median_province_price' not in df:
    #    df = pd.merge(df, groupby_province_price, on='province', how='left') 
    

    df['city_cat'] = df['city'].factorize()[0]
    df['county_cat'] = df['county'].factorize()[0]
    df['province_cat'] = df['province'].factorize()[0]
    
    
    cat_feats = {
        "materiał budynku": "build_material_cat",
        "okna": "window_cat",
        "stan wykończenia": "property_completion_cat",
        "rodzaj zabudowy": "property_type_cat",
        "ogrzewanie": "property_heating_cat",
        "forma własności": "own_property_cat"
     }    
    
    for feat_name, feat_new_name in cat_feats.items():
        df[feat_new_name] = df[feat_name].factorize()[0]
        
        #OHE
        df_dummies = pd.get_dummies(df[feat_name])
        df_dummies.columns = ['{0}_{1}'.format(feat_new_name, x) for x in df_dummies.columns]
        pd.concat([df, df_dummies], axis=1)
        
    stats = df['stats'].apply(pd.Series)
    df = pd.concat([df, stats], axis=1)
    dict_created_at={
        'ponad 14 dni temu':18,  
        '23.10.2018':17,
        '24.10.2018':16,
        '25.10.2018':15,
        '26.10.2018':14, 
        '27.10.2018':13,
        '28.10.2018':12,
        '29.10.2018':11,
        '30.10.2018':10,
        '31.10.2018':9,
        '01.11.2018':8,    
        '02.11.2018':7,
        '03.11.2018':6,
        '04.11.2018':5,
        '05.11.2018':4,
        '06.11.2018':3,
        '07.11.2018':2,
        '08.11.2018':1,
    }
        
    df['created_at_cat'] = df['created_at'].map(dict_created_at)
    #df['created_at_cat'] = df['created_at'].factorize()[0]
    #df['visit_ads_num'] = df.visit_ads.fillna(-1).astype('int')  
    df['visit_ads_num'] = df.visit_ads.map(lambda x: np.log(int(x) + 10) if not isinstance(x, float)  else -1)
    
    
    #Starter3
    #Location
    for i in range(5):
        df["loc{}".format(i)] = df["location"].map(lambda x: x[i] if len(x) > i else "")      
    
    df['loc01'] = df['loc0'] + df['loc1']
    df['loc012'] = df['loc0'] + df['loc1'] + df['loc2']
    df['loc12'] = df['loc1'] + df['loc2']
    
    #cat location
    for i in range(5):
        df["loc{}_cat".format(i)] = df["loc{}".format(i)].factorize()[0]
    df["loc01_cat"] = df['loc01'].factorize()[0]
    df["loc012_cat"] = df['loc012'].factorize()[0] 
    df["loc12_cat"] = df['loc12'].factorize()[0] 
    
    def groupby_location(df, loc, feat):
        groupby_feat = df_groupby_feat(df, [loc], feat)        
        if 'median_{}_price'.format(loc) not in df:
            return pd.merge(df, groupby_feat, on='{}'.format(loc), how='left')
    
    #df = groupby_location(df, 'loc01', 'price')    
    #df = groupby_location(df, 'loc12')
    
    #Starter 4
    
    df['security'] = df['system alarmowy'] | df['rolety antywłamaniowe'] | df['drzwi / okna antywłamaniowe']
    
    df['area_per_room'] = df['area_norm'] / df["rooms"]
    
    years = [1970, 1980, 1990, 2000, 2005, 2010, 2012, 2014, 2016, 2017]
    
    def build_year_norm(year):
        if year < 1970: return 1900
        if year > 2017: return 2018

        for idx in range(len(years) - 1):
            if years[idx+1] > year >= years[idx]:
                return years[idx]

    df["build_year_norm"] = df["build_year"].map(build_year_norm)

    
    big_cities = {'Poznań', 'Sopot', 'Wrocław', 'Kraków', 'Gdańsk', 'Gdynia', 'Opole', 'Katowice',  'Częstochowa', 'Szczecin', 'Kalisz', 'Łódź', 'Olsztyn', 'Warszawa'}
    for city in big_cities:
        df[city] = df['city'] == city
        df['big_city'] = df['city'].map(lambda x: x in big_cities)
        
    
    #location
    df_val_cnts = df['loc12'].value_counts()
    loc12_vals = set(df_val_cnts[ df_val_cnts > 100].index.values)
    for item in loc12_vals:
        df[item] = df['loc12'] == item
        
    
    #primary_market
    
    def is_primary_market_conc(df, feat):
        df['is_primary_market_{}'.format(feat)] = df[ ['is_primary_market', feat] ].apply(
            lambda x: '{}_{}'.format(x['is_primary_market'], x[feat]), axis=1
        )
        df['is_primary_market_{}_cat'.format(feat)] = df['is_primary_market_{}'.format(feat)].factorize()[0]

        return df
    
    df = is_primary_market_conc(df, 'rooms')
    df = is_primary_market_conc(df, 'city')
    df = is_primary_market_conc(df, 'rodzaj zabudowy') 
    
    #agregacje price_m2
    groupby_price_m2 = df_groupby_feat(df, ['is_primary_market_rooms'], 'price_m2')
    df = pd.merge(df, groupby_price_m2, on='is_primary_market_rooms', how='left')
    
    #groupby_price_m2 = df_groupby_feat(df, ['is_primary_market_city'], 'price_m2')
    #df = pd.merge(df, groupby_price_m2, on='is_primary_market_city', how='left')
    
    groupby_price_m2 = df_groupby_feat(df, ['is_primary_market_rodzaj zabudowy'], 'price_m2')
    df = pd.merge(df, groupby_price_m2, on='is_primary_market_rodzaj zabudowy', how='left')
    

                                                                   
    return df

(46489, 53) (46275, 52) (92764, 53)


<function gc.collect(generation=2)>

# BASIC MODEL - Kaggle Score: 182576.93612

In [ ]:
df = feature_engineering(df_org.copy())
print(df.shape)
df.sample(3)

In [ ]:
feats = ['id']
model = DummyRegressor()
model.fit(df_train[feats].values, df_train['price'])
y_pred = model.predict(df_test[feats].values)

df_test['price'] = y_pred
df_test[ ['id', 'price'] ].to_csv('../output/DummyRegressor.csv', index=False)

In [265]:
#msno.bar(df_train.iloc[:,40:50])

# ⭐ DAY 4 MODEL = Score: 51517.18706 Public score: 54286.18086

## 🤖 MLflow

In [ ]:
!mlflow ui --backend-store-uri file:///home/jovyan/dwsolutions/property_warsaw/notebooks/mlruns

In [56]:
!mlflow ui --backend-store-uri file:///home\kowsc\Dropbox\!Dev\!My projects\property_forecast_excel_deploy\mlruns

Usage: mlflow ui [OPTIONS]
Try 'mlflow ui --help' for help.

Error: Got unexpected extra argument (projects\property_forecast_excel_deploy\mlruns)


 ### [Dashboard](/hub/user-redirect/proxy/5000/) 

In [55]:
start_experiment(df, 'catboost_hyper', scoring=mean_absolute_error,
                 filename='catboost_hyper',
                 results=True,
                 export=True,
                 mlflow_save=True)

Working on "catboost_hyper_0" experiment


5it [04:53, 58.73s/it]


catboost_hyper_0.csv save successfully in "output" folder
Experiment catboost_hyper_0 recorded
(53379.28209332221, 2194.4865630331637)


Weight,Feature
0.1300,mean_city_price
0.0811,area_norm
0.0793,area_num_log
0.0727,median_city_price
0.0671,mean_county_price
0.0566,area_num
0.0371,build_year
0.0312,median_county_price
0.0259,visit_ads_num
0.0250,property_type_cat


In [54]:
start_experiment(df,
                 'catboost_light',
                 scoring=mean_absolute_error,
                 filename='catboost_light',
                 results=True,
                 export=True,
                 mlflow_save=True)

Working on "catboost_light_7" experiment


5it [00:36,  7.39s/it]


catboost_light_7.csv save successfully in "output" folder
Experiment catboost_light_7 recorded
(62696.83907477024, 1796.5045283911506)


Weight,Feature
0.1818,mean_county_price
0.1660,area_num_log
0.1546,mean_city_price
0.1101,area_norm
0.0376,median_county_price
0.0369,build_year
0.0355,area_num
0.0303,property_type_cat
0.0161,median_is_primary_market_rodzaj zabudowy_price_m2
0.0143,mean_is_primary_market_rodzaj zabudowy_price_m2
